In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import re

# Input: rcp_no

def get_report_attach_urls(rcp_no):
    
    excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
    
    attach_urls = []

    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)

    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]


    attach_urls = excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no)
    print(attach_urls)
    return attach_urls


import pandas as pd

url = get_report_attach_urls('20190401004781')

df = pd.read_excel(url, sheet_name='재무상태표', index_col=0, skiprows=6)



http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20190401004781&dcm_no=6616741&lang=ko


In [3]:
df.head(30)

,제 50 기,제 49 기,제 48 기
,,,
자산,,,
유동자산,80039455,70155189,69981128
현금및현금성자산,2607957,2763768,3778371
단기금융상품,34113871,25510064,30170656
매출채권,24933267,27881777,23514012
미수금,1515079,2201402,2319782
선급금,807262,1097598,814300
선급비용,2230628,2281179,2375520
재고자산,12440951,7837144,5981634


### 항목 분류컬럼생성

In [ ]:
# 공백개수로 분류하여 리스트화 : hier_num

##가정1. 재무상태표 항목입력 시 계층 상하관계에따라 공백을 달리 표시했다.

hier_num = sorted(set([ len(i) - len(i.lstrip()) for i in df2.index]))




# 컬럼 추가( hier_num 리스트 분류 만큼 )
a_col=[]
for i in range(len(hier_num)):

    name = '분류{}'.format(i)
    a_col.append(name)
    df2[name]= ''


### 항목 분류컬럼 값 채우기

In [ ]:

# 계층구조에 맞게 컬럼값 채우기

##가정1. 첫 행렬값이 가장 최상위 계층이다.
##가정2. 계정항목들은 상하관계에 따라 순차적으로 정렬되어있다.
##가정3: 하위계층을 가지는 항목을 가지는 데이터는 사용할 필요가 없다.

rm_idx_list = []


before = 0



for ix,i in enumerate(df2.index):
    
    
    
    a = len(i)-len(i.lstrip())
    
    #하위계층인지 체크
    if  a > before:
               
        rm_idx_list.append(ix-1)  # 해당 항목이 앞선항목의 하위항목에 해당된다면, 이전 항목의 row index를 저장하라
 

        #계층컬럼값 채우기
        for j in hier_num: 

            if j == a:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = i.strip()
                
                break


            else:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()
        
        
        
        
        before = a



    else:   
        
        
        #계층컬럼값 채우기
        for j in hier_num: 

            if j == a:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = i.strip()
                
                break


            else:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()
                
                
        
        before = a



### 하위계층을 더이상 갖지 않는 항목의 레코드만 추출 & 총계를 표시하는 레코드 제외

In [ ]:
set1 = set(range(len(df2))) 

set2 = set(rm_idx_list)

rows_to_use = list(set1.difference(set2))


df3 = df2.iloc[rows_to_use]
df3 = df3[df3.index.str.contains('총계') == False ]

In [ ]:
df3

### '분기' 컬럼을 로우로 변환

In [ ]:
df3.index.name = '항목명'

df4 = df3.reset_index()

df5 = pd.melt(df4, id_vars=['항목명','분류0','분류1','분류2'],var_name=['분기'])

df5 = df5.rename(columns={'value':'금액'})


### parent 컬럼 생성 및 값 채우기

In [97]:
#부모계층컬럼생성
df5['parent'] =''


for ix,row in df5[a_col].iterrows():

    rows= [x for x in row if x]
    df5['parent'][ix] = rows[-1]

<br><br><br><br>


# 통합 code

In [100]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import re

#################################################################
def get_report_attach_urls(rcp_no):
    
    excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
    
    attach_urls = []

    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)

    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]


    attach_urls = excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no)

    return attach_urls




def preprocessing(data,rm_idx_list,elem_list):
    
    global bs_table
    
    #최하위계층 항목만 추출
    set1 = set(range(len(data))) 

    set2 = set(rm_idx_list)

    rows_to_use = list(set1.difference(set2))
    
    df3 = data.iloc[rows_to_use]
    
    
    #'총계' 레코드 제외
    df3 = df3[df3.index.str.contains('총계') == False ]


    
    # 컬럼명을 컬럼값으로 내리기
    
    df3.index.name = '항목명'
    df4 = df3.reset_index()    
    
    id_vars = ['항목명']
    id_vars.extend(elem_list)   
    df5 = pd.melt(df4, id_vars= id_vars,var_name=['분기'])


    bs_table = df5.rename(columns={'value':'금액'})
    
    
    
#     #부모계층컬럼생성
#     bs_table['parent'] =''


#     for ix,row in bs_table[a_col].iterrows():

#         rows= [x for x in row if x]
#         bs_table['parent'][ix] = rows[-1]

        
        
##############################################################




url = get_report_attach_urls('20190401004781')  # 20190401004781 는 rcp_no , 문서접수번호(고유번호)

df = pd.read_excel(url, sheet_name='재무상태표', index_col=0, skiprows=6)

df2 =df




# 공백개수로 분류하여 리스트화 : hier_num

##가정1. 재무상태표 항목입력 시 계층 상하관계에따라 공백을 달리 표시했다.

hier_num = sorted(set([ len(i) - len(i.lstrip()) for i in df2.index]))



# 컬럼 추가( hier_num 리스트 분류 만큼 )
a_col=[]
for i in range(len(hier_num)):

    name = '분류{}'.format(i+1)
    a_col.append(name)
    df2[name]= ''

    
    
    
# 계층구조에 맞게 컬럼값 채우기

##가정1. 첫 행렬값이 가장 최상위 계층이다.
##가정2. 계정항목들은 상하관계에 따라 순차적으로 정렬되어있다.
##가정3: 하위계층을 가지는 항목을 가지는 데이터는 사용할 필요가 없다.

rm_idx_list = []


before = 0



for ix,i in enumerate(df2.index):
    
    
    
    a = len(i)-len(i.lstrip())
    
    #하위계층인지 체크
    if  a > before:
               
        rm_idx_list.append(ix-1)  # 해당 항목이 앞선항목의 하위항목에 해당된다면, 이전 항목의 row index를 저장하라
 

        #계층컬럼값 채우기
        for j in hier_num: 

            if j == a:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = i.strip()
                
                break


            else:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()
        
        
        
        
        before = a



    else:   
        
        
        #계층컬럼값 채우기
        for j in hier_num: 

            if j == a:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = i.strip()
                
                break


            else:

                b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()
                
                
        
        before = a
        
        
        

        

# 실행 : output -> DB에 저장될  " bs_table ""  생성


preprocessing(df2,rm_idx_list,a_col)




# 함수화시키기

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import re

#################################################################

    
    
    
#     #부모계층컬럼생성
#     bs_table['parent'] =''


#     for ix,row in bs_table[a_col].iterrows():

#         rows= [x for x in row if x]
#         bs_table['parent'][ix] = rows[-1]

        
        
##############################################################

def BS_TABLE(rcp_no):
    
    global bs_table
    
    def get_report_attach_urls(rcp_no):
    
        excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"

        attach_urls = []

        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
        r = requests.get(url)

        start_str = "javascript: viewDoc\('" + rcp_no + "', '"
        end_str = "', null, null, null,"
        reg = re.compile(start_str + '(.*)' + end_str)
        m = reg.findall(r.text)
        dcm_no = m[0]


        attach_urls = excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no)

        return attach_urls




    def preprocessing(data,rm_idx_list,elem_list):

        global bs_table

        #최하위계층 항목만 추출
        set1 = set(range(len(data))) 

        set2 = set(rm_idx_list)

        rows_to_use = list(set1.difference(set2))

        df3 = data.iloc[rows_to_use]


        #'총계' 레코드 제외
        df3 = df3[df3.index.str.contains('총계') == False ]



        # 컬럼명을 컬럼값으로 내리기

        df3.index.name = '항목명'
        df4 = df3.reset_index()    

        id_vars = ['항목명']
        id_vars.extend(elem_list)   
        df5 = pd.melt(df4, id_vars= id_vars,var_name=['분기'])


        bs_table = df5.rename(columns={'value':'금액'})


        
        
        
    url = get_report_attach_urls(rcp_no)  # 20190401004781 는 rcp_no , 문서접수번호(고유번호)
    


    df = pd.read_excel(url, sheet_name= '재무상태표', index_col=0, skiprows=6)




    # 공백개수로 분류하여 리스트화 : hier_num

    ##가정1. 재무상태표 항목입력 시 계층 상하관계에따라 공백을 달리 표시했다.

    hier_num = sorted(set([ len(i) - len(i.lstrip()) for i in df2.index]))



    # 컬럼 추가( hier_num 리스트 분류 만큼 )
    a_col=[]
    for i in range(len(hier_num)):

        name = '분류{}'.format(i+1)
        a_col.append(name)
        df2[name]= ''




    # 계층구조에 맞게 컬럼값 채우기

    ##가정1. 첫 행렬값이 가장 최상위 계층이다.
    ##가정2. 계정항목들은 상하관계에 따라 순차적으로 정렬되어있다.
    ##가정3: 하위계층을 가지는 항목을 가지는 데이터는 사용할 필요가 없다.

    rm_idx_list = []


    before = 0



    for ix,i in enumerate(df2.index):



        a = len(i)-len(i.lstrip())

        #하위계층인지 체크
        if  a > before:

            rm_idx_list.append(ix-1)  # 해당 항목이 앞선항목의 하위항목에 해당된다면, 이전 항목의 row index를 저장하라


            #계층컬럼값 채우기
            for j in hier_num: 

                if j == a:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = i.strip()

                    break


                else:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()




            before = a



        else:   


            #계층컬럼값 채우기
            for j in hier_num: 

                if j == a:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = i.strip()

                    break


                else:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()



            before = a






    # 실행 : output -> DB에 저장될  " bs_table ""  생성


    preprocessing(df2,rm_idx_list,a_col)



    
    
    
BS_TABLE('20190401004781')

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import re

#################################################################



def IS_TABLE(rcp_no):
    
    global bs_table
    
    def get_report_attach_urls(rcp_no):
    
        excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"

        attach_urls = []

        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
        r = requests.get(url)

        start_str = "javascript: viewDoc\('" + rcp_no + "', '"
        end_str = "', null, null, null,"
        reg = re.compile(start_str + '(.*)' + end_str)
        m = reg.findall(r.text)
        dcm_no = m[0]


        attach_urls = excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no)

        return attach_urls




    def preprocessing(data,rm_idx_list,elem_list):

        global bs_table

        #최하위계층 항목만 추출
        set1 = set(range(len(data))) 

        set2 = set(rm_idx_list)

        rows_to_use = list(set1.difference(set2))

        df3 = data.iloc[rows_to_use]


        #'총계' 레코드 제외
        df3 = df3[df3.index.str.contains('총계') == False ]



        # 컬럼명을 컬럼값으로 내리기

        df3.index.name = '항목명'
        df4 = df3.reset_index()    

        id_vars = ['항목명']
        id_vars.extend(elem_list)   
        df5 = pd.melt(df4, id_vars= id_vars,var_name=['분기'])


        bs_table = df5.rename(columns={'value':'금액'})


        
        
        
    url = get_report_attach_urls(rcp_no)  # 20190401004781 는 rcp_no , 문서접수번호(고유번호)
    


    df = pd.read_excel(url, sheet_name= '재무상태표', index_col=0, skiprows=6)




    # 공백개수로 분류하여 리스트화 : hier_num

    ##가정1. 재무상태표 항목입력 시 계층 상하관계에따라 공백을 달리 표시했다.

    hier_num = sorted(set([ len(i) - len(i.lstrip()) for i in df2.index]))



    # 컬럼 추가( hier_num 리스트 분류 만큼 )
    a_col=[]
    for i in range(len(hier_num)):

        name = '분류{}'.format(i+1)
        a_col.append(name)
        df2[name]= ''




    # 계층구조에 맞게 컬럼값 채우기

    ##가정1. 첫 행렬값이 가장 최상위 계층이다.
    ##가정2. 계정항목들은 상하관계에 따라 순차적으로 정렬되어있다.
    ##가정3: 하위계층을 가지는 항목을 가지는 데이터는 사용할 필요가 없다.

    rm_idx_list = []


    before = 0



    for ix,i in enumerate(df2.index):



        a = len(i)-len(i.lstrip())

        #하위계층인지 체크
        if  a > before:

            rm_idx_list.append(ix-1)  # 해당 항목이 앞선항목의 하위항목에 해당된다면, 이전 항목의 row index를 저장하라


            #계층컬럼값 채우기
            for j in hier_num: 

                if j == a:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = i.strip()

                    break


                else:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()




            before = a



        else:   


            #계층컬럼값 채우기
            for j in hier_num: 

                if j == a:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = i.strip()

                    break


                else:

                    b = df2.columns.tolist().index(a_col[hier_num.index(j)])
                    df2.iloc[ix ,b] = df2.iloc[ix-1,b].strip()



            before = a






    # 실행 : output -> DB에 저장될  " bs_table ""  생성


    preprocessing(df2,rm_idx_list,a_col)



    
    
    
BS_TABLE('20190401004781')

,항목명,분류1,분류2,분기,금액
0,매출원가,매출원가,,170381870,101666506
1,매출총이익,매출총이익,,170381870,68715364
2,판매비와관리비,판매비와관리비,,170381870,25015913
3,영업이익(손실),영업이익(손실),,170381870,43699451
4,기타수익,기타수익,,170381870,972145
5,기타비용,기타비용,,170381870,504562
6,금융수익,금융수익,,170381870,3737494
7,금융비용,금융비용,,170381870,3505673
8,법인세비용차감전순이익(손실),법인세비용차감전순이익(손실),,170381870,44398855
9,법인세비용,법인세비용,,170381870,11583728


In [112]:
bs_table.to_csv('C:/DATA/BS_TABLE1.csv',index=True , header =True,encoding='ms949')

In [103]:
import requests
from bs4 import BeautifulSoup
import os
import time
import telegram
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import re

#################################################################
def get_report_attach_urls(rcp_no):
    
    excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
    
    attach_urls = []

    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)

    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]


    attach_urls = excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no)

    return attach_urls



url = get_report_attach_urls('20190401004781')  # 20190401004781 는 rcp_no , 문서접수번호(고유번호)

# df = pd.read_excel(url, sheet_name='재무상태표', index_col=0, skiprows=6)

# df = pd.read_excel(url)

xl = pd.ExcelFile(url)

xl.sheet_names


# df.index.name = '항목명'
# df2 = df.reset_index()    


# df3 = pd.melt(df2,id_vars=['항목명'], var_name=['분기'])


# df3.to_csv('C:/DATA/df3.csv',index=True , header =True,encoding='ms949')

# df2



['기본정보',
 '연결 재무상태표',
 '연결 손익계산서',
 '연결 포괄손익계산서',
 '연결 자본변동표',
 '연결 현금흐름표',
 '재무상태표',
 '손익계산서',
 '포괄손익계산서',
 '자본변동표',
 '현금흐름표']

In [99]:
df3

,항목명,분기,value
0,자산,제 50 기,
1,유동자산,제 50 기,80039455
2,현금및현금성자산,제 50 기,2607957
3,단기금융상품,제 50 기,34113871
4,매출채권,제 50 기,24933267
5,미수금,제 50 기,1515079
6,선급금,제 50 기,807262
7,선급비용,제 50 기,2230628
8,재고자산,제 50 기,12440951
9,기타유동자산,제 50 기,1390440
